# Environmental Data Retriever

This notebook consolidates soil and weather data retrieval functions into a systematic and organized structure. It combines the functionality from the last 4 blocks of Untitled22.ipynb into clear, reusable functions.

## 1. Import Required Libraries

Import all necessary libraries and dependencies required for the data retrieval functions.

In [1]:
import requests
import json
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any

print("✅ All required libraries imported successfully!")

✅ All required libraries imported successfully!


## 2. Define Data Retrieval Functions

Create individual functions from each of the 4 code blocks, ensuring proper function signatures, documentation, and error handling.

In [2]:
def extract_soil_properties(filename: str = "soil_data.json", depth_label: str = "0-5cm") -> Dict[str, Any]:
    """
    Extract specific soil properties: texture, nitrogen, organic carbon, and pH
    
    Args:
        filename (str): Path to the soil data JSON file
        depth_label (str): Soil depth layer to extract data from
    
    Returns:
        Dict[str, Any]: Dictionary containing soil properties
    """
    try:
        with open(filename, "r") as f:
            data = json.load(f)
        
        layers = data["properties"]["layers"]
        
        def get_property_value(layer_name: str) -> Optional[float]:
            """Extract mean value for a specific property at given depth"""
            for layer in layers:
                if layer["name"] == layer_name:
                    d_factor = layer["unit_measure"].get("d_factor", 1)
                    for depth in layer["depths"]:
                        if depth["label"] == depth_label:
                            mean_val = depth["values"].get("mean")
                            if mean_val is not None:
                                return mean_val / d_factor
            return None
        
        # Extract individual soil components
        soil_properties = {
            "ph": get_property_value("phh2o"),
            "nitrogen": get_property_value("nitrogen"),
            "organic_carbon": get_property_value("soc"),
            "sand_percent": get_property_value("sand"),
            "clay_percent": get_property_value("clay"),
            "silt_percent": get_property_value("silt")
        }
        
        # Determine soil texture class
        soil_properties["texture_type"] = determine_soil_texture(
            soil_properties["sand_percent"],
            soil_properties["clay_percent"],
            soil_properties["silt_percent"]
        )
        
        return soil_properties
        
    except FileNotFoundError:
        print(f"❌ Soil data file '{filename}' not found")
        return {}
    except Exception as e:
        print(f"❌ Error extracting soil properties: {e}")
        return {}


def determine_soil_texture(sand_pct: Optional[float], clay_pct: Optional[float], silt_pct: Optional[float]) -> str:
    """
    Determine soil texture class based on sand, clay, silt percentages using USDA classification
    
    Args:
        sand_pct (Optional[float]): Sand percentage
        clay_pct (Optional[float]): Clay percentage
        silt_pct (Optional[float]): Silt percentage
    
    Returns:
        str: Soil texture classification
    """
    if None in [sand_pct, clay_pct, silt_pct]:
        return "unknown"
    
    # USDA Soil Texture Classification
    if clay_pct >= 40:
        return "clay"
    elif clay_pct >= 27:
        if sand_pct > 45:
            return "sandy_clay"
        elif sand_pct > 20:
            return "clay_loam"
        else:
            return "silty_clay"
    elif clay_pct >= 20:
        if sand_pct > 45:
            return "sandy_clay_loam"
        elif sand_pct > 20:
            return "clay_loam"
        else:
            return "silt_loam"
    elif silt_pct >= 50:
        if clay_pct < 12 and sand_pct < 50:
            return "silt"
        else:
            return "silt_loam"
    elif sand_pct >= 70:
        if clay_pct < 15:
            return "sand"
        else:
            return "sandy_loam"
    else:
        return "loam"


def get_key_soil_properties(filename: str = "soil_data.json") -> Dict[str, Any]:
    """
    Simplified function to extract only the four key soil properties:
    - texture_type
    - nitrogen
    - organic_carbon  
    - ph
    
    Args:
        filename (str): Path to the soil data JSON file
    
    Returns:
        Dict[str, Any]: Dictionary containing key soil properties
    """
    full_properties = extract_soil_properties(filename)
    
    if not full_properties:
        return {}
    
    key_properties = {
        "texture_type": full_properties.get("texture_type", "unknown"),
        "nitrogen": full_properties.get("nitrogen"),
        "organic_carbon": full_properties.get("organic_carbon"),
        "ph": full_properties.get("ph")
    }
    
    return key_properties


def get_weather_data(lat: float, lon: float, api_key: Optional[str] = None) -> Optional[Dict[str, Any]]:
    """
    Fetch current weather data from OpenWeather API
    Returns temperature and humidity for the given coordinates
    
    Args:
        lat (float): Latitude coordinate
        lon (float): Longitude coordinate
        api_key (Optional[str]): OpenWeather API key
    
    Returns:
        Optional[Dict[str, Any]]: Weather data dictionary or None if failed
    """
    # Default API key (replace with your actual key)
    if api_key is None:
        api_key = os.getenv("OPENWEATHER_API_KEY", "your_api_key_here")
    
    if not api_key or api_key == "your_api_key_here":
        print("⚠️ Please set your OpenWeather API key!")
        print("Get a free API key from: https://openweathermap.org/api")
        # Return sample data for demonstration
        return {
            "temperature": 25.5,
            "humidity": 65,
            "description": "Sample data (API key needed)",
            "location": f"Lat: {lat}, Lon: {lon}",
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
    
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "lat": lat,
        "lon": lon,
        "appid": api_key,
        "units": "metric"  # For Celsius temperature
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        weather_data = {
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "description": data["weather"][0]["description"],
            "location": data["name"],
            "country": data["sys"]["country"],
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "feels_like": data["main"]["feels_like"],
            "pressure": data["main"]["pressure"],
            "wind_speed": data.get("wind", {}).get("speed", 0)
        }
        
        return weather_data
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Weather API request failed: {e}")
        return None
    except Exception as e:
        print(f"❌ Error processing weather data: {e}")
        return None


def get_weather_for_coordinates(coords_list: List[Tuple[float, float]]) -> Tuple[Optional[Dict[str, Any]], float, float]:
    """
    Get weather data for the same coordinates used for soil data
    Uses the average coordinates approach from the soil data fetching
    
    Args:
        coords_list (List[Tuple[float, float]]): List of (latitude, longitude) tuples
    
    Returns:
        Tuple[Optional[Dict[str, Any]], float, float]: Weather data, average lat, average lon
    """
    if not coords_list:
        raise ValueError("Coordinate list is empty")

    avg_lat = sum(lat for lat, lon in coords_list) / len(coords_list)
    avg_lon = sum(lon for lat, lon in coords_list) / len(coords_list)
    
    print(f"🌍 Getting weather data for average coordinates:")
    print(f"   Latitude: {avg_lat:.4f}, Longitude: {avg_lon:.4f}")
    
    weather_data = get_weather_data(avg_lat, avg_lon)
    return weather_data, avg_lat, avg_lon

print("✅ All data retrieval functions defined successfully!")

✅ All data retrieval functions defined successfully!


## 3. Create Main Data Retriever Class

Organize all functions into a cohesive DataRetriever class that encapsulates the functionality from the original blocks.

In [3]:
class EnvironmentalDataRetriever:
    """
    A comprehensive class for retrieving and combining soil and weather data
    for environmental analysis and agricultural applications.
    """
    
    def __init__(self, openweather_api_key: Optional[str] = None):
        """
        Initialize the EnvironmentalDataRetriever
        
        Args:
            openweather_api_key (Optional[str]): OpenWeather API key
        """
        self.openweather_api_key = openweather_api_key or os.getenv("OPENWEATHER_API_KEY")
        self.default_coordinates = [
            (28.6139, 77.2090),  # Delhi
            (19.0760, 72.8777),  # Mumbai  
            (13.0827, 80.2707)   # Chennai
        ]
    
    def get_soil_properties(self, filename: str = "soil_data.json", 
                           depth_label: str = "0-5cm", 
                           key_properties_only: bool = True) -> Dict[str, Any]:
        """
        Extract soil properties from soil data file
        
        Args:
            filename (str): Path to soil data JSON file
            depth_label (str): Soil depth layer to extract
            key_properties_only (bool): If True, return only key properties
        
        Returns:
            Dict[str, Any]: Soil properties data
        """
        if key_properties_only:
            return get_key_soil_properties(filename)
        else:
            return extract_soil_properties(filename, depth_label)
    
    def get_weather_conditions(self, coords_list: Optional[List[Tuple[float, float]]] = None) -> Tuple[Optional[Dict[str, Any]], float, float]:
        """
        Get weather conditions for given coordinates
        
        Args:
            coords_list (Optional[List[Tuple[float, float]]]): List of coordinates
        
        Returns:
            Tuple[Optional[Dict[str, Any]], float, float]: Weather data, lat, lon
        """
        if coords_list is None:
            coords_list = self.default_coordinates
        
        return get_weather_for_coordinates(coords_list)
    
    def get_combined_environmental_data(self, 
                                       coords_list: Optional[List[Tuple[float, float]]] = None,
                                       soil_filename: str = "soil_data.json") -> Dict[str, Any]:
        """
        Combine soil and weather data for comprehensive environmental analysis
        Returns both soil properties and current weather conditions
        
        Args:
            coords_list (Optional[List[Tuple[float, float]]]): List of coordinates
            soil_filename (str): Path to soil data file
        
        Returns:
            Dict[str, Any]: Combined environmental data
        """
        if coords_list is None:
            coords_list = self.default_coordinates
        
        # Get soil data
        soil_data = self.get_soil_properties(soil_filename, key_properties_only=True)
        
        # Get weather data
        weather_data, lat, lon = self.get_weather_conditions(coords_list)
        
        # Combine data
        environmental_data = {
            "coordinates": {
                "latitude": lat,
                "longitude": lon
            },
            "soil": soil_data,
            "weather": {
                "temperature": weather_data['temperature'] if weather_data else None,
                "humidity": weather_data['humidity'] if weather_data else None,
                "description": weather_data.get('description', 'N/A') if weather_data else 'N/A',
                "feels_like": weather_data.get('feels_like') if weather_data else None,
                "pressure": weather_data.get('pressure') if weather_data else None,
                "wind_speed": weather_data.get('wind_speed') if weather_data else None
            },
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return environmental_data
    
    def print_environmental_summary(self, environmental_data: Dict[str, Any]) -> None:
        """
        Print a formatted summary of environmental data
        
        Args:
            environmental_data (Dict[str, Any]): Combined environmental data
        """
        print("🌱 === ENVIRONMENTAL DATA SUMMARY ===")
        print(f"📍 Location: {environmental_data['coordinates']['latitude']:.4f}, {environmental_data['coordinates']['longitude']:.4f}")
        print(f"📅 Timestamp: {environmental_data['timestamp']}")
        print()
        
        # Weather summary
        weather = environmental_data['weather']
        print("🌡️ WEATHER CONDITIONS:")
        print(f"   Temperature: {weather['temperature']}°C")
        print(f"   Humidity: {weather['humidity']}%")
        print(f"   Conditions: {weather['description'].title()}")
        if weather.get('feels_like'):
            print(f"   Feels Like: {weather['feels_like']}°C")
        if weather.get('pressure'):
            print(f"   Pressure: {weather['pressure']} hPa")
        if weather.get('wind_speed'):
            print(f"   Wind Speed: {weather['wind_speed']} m/s")
        print()
        
        # Soil summary
        soil = environmental_data['soil']
        print("🌱 SOIL PROPERTIES:")
        if soil:
            print(f"   Texture Type: {soil.get('texture_type', 'Unknown')}")
            if soil.get('ph'):
                print(f"   pH: {soil['ph']:.2f}")
            if soil.get('nitrogen'):
                print(f"   Nitrogen: {soil['nitrogen']:.2f} g/kg")
            if soil.get('organic_carbon'):
                print(f"   Organic Carbon: {soil['organic_carbon']:.2f} g/kg")
        else:
            print("   No soil data available")

print("✅ EnvironmentalDataRetriever class created successfully!")

✅ EnvironmentalDataRetriever class created successfully!


## 4. Test the Consolidated Functions

Execute test cases to verify that the consolidated functions work correctly and produce expected outputs.

In [4]:
# Test the EnvironmentalDataRetriever class
print("🧪 === TESTING ENVIRONMENTAL DATA RETRIEVER ===")
print()

# Initialize the data retriever
data_retriever = EnvironmentalDataRetriever()

# Test coordinates (same as original)
test_coordinates = [
    (28.6139, 77.2090),  # Delhi
    (19.0760, 72.8777),  # Mumbai  
    (13.0827, 80.2707)   # Chennai
]

print("1️⃣ Testing soil properties extraction:")
try:
    soil_data = data_retriever.get_soil_properties()
    if soil_data:
        print("   ✅ Soil data extracted successfully")
        for key, value in soil_data.items():
            if isinstance(value, (int, float)):
                print(f"   📊 {key}: {value:.2f}")
            else:
                print(f"   📊 {key}: {value}")
    else:
        print("   ⚠️ No soil data available (check if soil_data.json exists)")
except Exception as e:
    print(f"   ❌ Soil data extraction failed: {e}")

print()
print("2️⃣ Testing weather data retrieval:")
try:
    weather_data, lat, lon = data_retriever.get_weather_conditions(test_coordinates)
    if weather_data:
        print("   ✅ Weather data retrieved successfully")
        print(f"   🌍 Coordinates: {lat:.4f}, {lon:.4f}")
        print(f"   🌡️ Temperature: {weather_data['temperature']}°C")
        print(f"   💧 Humidity: {weather_data['humidity']}%")
        print(f"   ☁️ Description: {weather_data['description']}")
    else:
        print("   ❌ Weather data retrieval failed")
except Exception as e:
    print(f"   ❌ Weather data retrieval failed: {e}")

print()
print("3️⃣ Testing combined environmental data:")
try:
    combined_data = data_retriever.get_combined_environmental_data(test_coordinates)
    print("   ✅ Combined data generated successfully")
    print()
    data_retriever.print_environmental_summary(combined_data)
except Exception as e:
    print(f"   ❌ Combined data generation failed: {e}")

print()
print("🎉 === TESTING COMPLETE ===")
print("📝 The EnvironmentalDataRetriever class successfully consolidates")
print("   all functionality from the original 4 code blocks into a")
print("   systematic and reusable structure!")

🧪 === TESTING ENVIRONMENTAL DATA RETRIEVER ===

1️⃣ Testing soil properties extraction:
   ✅ Soil data extracted successfully
   📊 texture_type: clay
   📊 nitrogen: 1.28
   📊 organic_carbon: 11.90
   📊 ph: 7.10

2️⃣ Testing weather data retrieval:
🌍 Getting weather data for average coordinates:
   Latitude: 20.2575, Longitude: 76.7858
⚠️ Please set your OpenWeather API key!
Get a free API key from: https://openweathermap.org/api
   ✅ Weather data retrieved successfully
   🌍 Coordinates: 20.2575, 76.7858
   🌡️ Temperature: 25.5°C
   💧 Humidity: 65%
   ☁️ Description: Sample data (API key needed)

3️⃣ Testing combined environmental data:
🌍 Getting weather data for average coordinates:
   Latitude: 20.2575, Longitude: 76.7858
⚠️ Please set your OpenWeather API key!
Get a free API key from: https://openweathermap.org/api
   ✅ Combined data generated successfully

🌱 === ENVIRONMENTAL DATA SUMMARY ===
📍 Location: 20.2575, 76.7858
📅 Timestamp: 2025-08-23 01:56:56

🌡️ WEATHER CONDITIONS:
   Te

## Usage Examples

Here are some practical usage examples of the consolidated functions:

In [5]:
# Example 1: Quick environmental data retrieval
print("📋 === USAGE EXAMPLES ===")
print()

# Initialize with custom API key (optional)
# retriever = EnvironmentalDataRetriever(openweather_api_key="your_api_key_here")
retriever = EnvironmentalDataRetriever()

# Example 1: Get data for custom coordinates
custom_coords = [(25.2048, 55.2708)]  # Dubai coordinates
print("1️⃣ Custom location example (Dubai):")
try:
    dubai_data = retriever.get_combined_environmental_data(custom_coords)
    print(f"   📍 Location: {dubai_data['coordinates']['latitude']:.4f}, {dubai_data['coordinates']['longitude']:.4f}")
    print(f"   🌡️ Temperature: {dubai_data['weather']['temperature']}°C")
    print(f"   💧 Humidity: {dubai_data['weather']['humidity']}%")
except Exception as e:
    print(f"   ❌ Error: {e}")

print()
print("2️⃣ Individual function usage:")
# Get only soil data
try:
    soil_only = retriever.get_soil_properties(key_properties_only=True)
    print(f"   🌱 Soil texture: {soil_only.get('texture_type', 'Unknown')}")
    print(f"   🧪 pH level: {soil_only.get('ph', 'Unknown')}")
except Exception as e:
    print(f"   ❌ Soil data error: {e}")

# Get only weather data
try:
    weather_only, lat, lon = retriever.get_weather_conditions()
    if weather_only:
        print(f"   🌤️ Weather: {weather_only['description']} at {weather_only['temperature']}°C")
except Exception as e:
    print(f"   ❌ Weather data error: {e}")

print()
print("✨ The consolidated functions provide a clean, systematic approach")
print("   to environmental data retrieval with proper error handling")
print("   and organized structure!")

📋 === USAGE EXAMPLES ===

1️⃣ Custom location example (Dubai):
🌍 Getting weather data for average coordinates:
   Latitude: 25.2048, Longitude: 55.2708
⚠️ Please set your OpenWeather API key!
Get a free API key from: https://openweathermap.org/api
   📍 Location: 25.2048, 55.2708
   🌡️ Temperature: 25.5°C
   💧 Humidity: 65%

2️⃣ Individual function usage:
   🌱 Soil texture: clay
   🧪 pH level: 7.1
🌍 Getting weather data for average coordinates:
   Latitude: 20.2575, Longitude: 76.7858
⚠️ Please set your OpenWeather API key!
Get a free API key from: https://openweathermap.org/api
   🌤️ Weather: Sample data (API key needed) at 25.5°C

✨ The consolidated functions provide a clean, systematic approach
   to environmental data retrieval with proper error handling
   and organized structure!


## JSON Export for ML Pipeline

Export the environmental data as JSON files for use in the machine learning prediction pipeline.

In [6]:
def export_environmental_data_to_json(retriever, coords_list=None, output_dir="crop_prediction_pipeline/data_output"):
    """
    Export environmental data to JSON files for ML pipeline consumption
    
    Args:
        retriever: EnvironmentalDataRetriever instance
        coords_list: List of coordinates (optional)
        output_dir: Directory to save JSON files
    
    Returns:
        dict: Paths to exported JSON files
    """
    import os
    from datetime import datetime
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get environmental data
    try:
        environmental_data = retriever.get_combined_environmental_data(coords_list)
        
        # Format data for ML model consumption
        formatted_data = {
            "metadata": {
                "timestamp": environmental_data["timestamp"],
                "coordinates": environmental_data["coordinates"],
                "data_source": "EnvironmentalDataRetriever"
            },
            "soil_data": environmental_data["soil"],
            "weather_data": environmental_data["weather"]
        }
        
        # Add NPK values (required for ML model)
        if formatted_data["soil_data"].get("texture_type"):
            # Map texture types to typical NPK values
            texture_npk_map = {
                "clay": {"N": 80, "P": 40, "K": 45},
                "sandy": {"N": 50, "P": 25, "K": 30},
                "loam": {"N": 70, "P": 35, "K": 35},
                "silt": {"N": 75, "P": 38, "K": 40},
                "sandy_loam": {"N": 60, "P": 30, "K": 32},
                "clay_loam": {"N": 75, "P": 38, "K": 42},
                "silt_loam": {"N": 72, "P": 36, "K": 38},
                "sandy_clay": {"N": 65, "P": 32, "K": 38},
                "silty_clay": {"N": 78, "P": 39, "K": 43},
                "sandy_clay_loam": {"N": 68, "P": 34, "K": 36}
            }
            texture = formatted_data["soil_data"]["texture_type"]
            formatted_data["soil_data"]["npk_values"] = texture_npk_map.get(texture, {"N": 70, "P": 35, "K": 35})
        
        # Add forecast data structure for ML model
        formatted_data["weather_data"]["forecast"] = {
            "avg_temperature": formatted_data["weather_data"]["temperature"],
            "avg_humidity": formatted_data["weather_data"]["humidity"],
            "total_rainfall_7days": 25.0  # Default rainfall (mm)
        }
        
        # Export to JSON files
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Combined environmental data
        env_file = os.path.join(output_dir, f"environmental_data_{timestamp}.json")
        with open(env_file, 'w') as f:
            json.dump(formatted_data, f, indent=2, ensure_ascii=False)
        
        # Separate soil data file
        soil_file = os.path.join(output_dir, f"soil_data_{timestamp}.json")
        with open(soil_file, 'w') as f:
            json.dump(formatted_data["soil_data"], f, indent=2, ensure_ascii=False)
        
        # Separate weather data file
        weather_file = os.path.join(output_dir, f"weather_data_{timestamp}.json")
        with open(weather_file, 'w') as f:
            json.dump(formatted_data["weather_data"], f, indent=2, ensure_ascii=False)
        
        # ML-ready combined data (flattened structure)
        ml_data = {
            "N": formatted_data["soil_data"]["npk_values"]["N"],
            "P": formatted_data["soil_data"]["npk_values"]["P"],
            "K": formatted_data["soil_data"]["npk_values"]["K"],
            "temperature": formatted_data["weather_data"]["temperature"] or 25.0,
            "humidity": formatted_data["weather_data"]["humidity"] or 65.0,
            "ph": formatted_data["soil_data"]["ph"] or 6.5,
            "rainfall": formatted_data["weather_data"]["forecast"]["total_rainfall_7days"],
            "metadata": formatted_data["metadata"]
        }
        
        ml_file = os.path.join(output_dir, f"ml_ready_data_{timestamp}.json")
        with open(ml_file, 'w') as f:
            json.dump(ml_data, f, indent=2, ensure_ascii=False)
        
        export_paths = {
            "environmental_data": env_file,
            "soil_data": soil_file,
            "weather_data": weather_file,
            "ml_ready_data": ml_file
        }
        
        print("📁 === JSON EXPORT SUCCESSFUL ===")
        print(f"✅ Environmental data: {env_file}")
        print(f"✅ Soil data: {soil_file}")
        print(f"✅ Weather data: {weather_file}")
        print(f"✅ ML-ready data: {ml_file}")
        
        return export_paths
        
    except Exception as e:
        print(f"❌ Error exporting environmental data: {e}")
        return {}

# Export current environmental data to JSON files
print("🔄 Exporting environmental data to JSON files...")
export_paths = export_environmental_data_to_json(retriever)

# Display the ML-ready data structure
if export_paths.get("ml_ready_data"):
    print("\n🤖 === ML-READY DATA STRUCTURE ===")
    with open(export_paths["ml_ready_data"], 'r') as f:
        ml_data = json.load(f)
    
    print("Model input features:")
    for key, value in ml_data.items():
        if key != "metadata":
            print(f"   {key}: {value}")
    
    print(f"\n📍 Location: {ml_data['metadata']['coordinates']['latitude']:.4f}, {ml_data['metadata']['coordinates']['longitude']:.4f}")
    print(f"⏰ Generated: {ml_data['metadata']['timestamp']}")

print("\n🎯 JSON files are ready for ML pipeline consumption!")

🔄 Exporting environmental data to JSON files...
🌍 Getting weather data for average coordinates:
   Latitude: 20.2575, Longitude: 76.7858
⚠️ Please set your OpenWeather API key!
Get a free API key from: https://openweathermap.org/api
📁 === JSON EXPORT SUCCESSFUL ===
✅ Environmental data: crop_prediction_pipeline/data_output/environmental_data_20250823_015656.json
✅ Soil data: crop_prediction_pipeline/data_output/soil_data_20250823_015656.json
✅ Weather data: crop_prediction_pipeline/data_output/weather_data_20250823_015656.json
✅ ML-ready data: crop_prediction_pipeline/data_output/ml_ready_data_20250823_015656.json

🤖 === ML-READY DATA STRUCTURE ===
Model input features:
   N: 80
   P: 40
   K: 45
   temperature: 25.5
   humidity: 65
   ph: 7.1
   rainfall: 25.0

📍 Location: 20.2575, 76.7858
⏰ Generated: 2025-08-23 01:56:56

🎯 JSON files are ready for ML pipeline consumption!
